In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir('../') # set the working directory to the root of the project
from symai import *
from IPython.display import display
from examples.sql import SQL
sql = SQL()

## SQL Example

The `SQL` expression is defined and passes as a global context the syntax of the SQL language. The SQL expression is then used to generate queries based on the given context. We can then use the generated queries to get data from a database.

The following query is sent to the neural computation engine and creates a query based on the given context:

In [27]:
q = sql('Select all users above the age of 30 from the Members table.'); display(q)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30)

We can now try to further manipulate the result by asking the model to incorporate additional information, such as filtering to a specific time range:

In [4]:
res = q << 'limit the query to the last 10 minutes'
display(res)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30 AND timestamp > NOW() - INTERVAL 10 MINUTE;)

We can also try to remove unwanted fields from the generated query. Notice how the model tries to remove not only the given statement but attributes associated with them:

In [5]:
res -= ' AND ...'
display(res)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30;)

And we can now even convert our query to other familiar domain specific languages, such as `SQL` or `ActiveRecord`:

In [6]:
sql_res = res.convert("ActiveRecord")
display(sql_res)

<class 'examples.sql.SQL'>(value=Member.where("age > ?", 30).select("*"))

To wrap things up, we might want to go back full circle and ask the model to generate again the explanation based on the given query:

In [7]:
answer_doc = res.query("What does this query do?")
display(answer_doc)

<class 'examples.sql.SQL'>(value=This query selects all data from the Members table where the age is greater than 30.)

Ask it in natural language to modify the query:

In [8]:
answer = res.query("How can you limit the number of results to 30 for an SQL query?")
display(answer)

<class 'examples.sql.SQL'>(value=Answer: 
You can limit the number of results to 30 for an SQL query by adding the LIMIT clause at the end of the query, like this: 
SELECT * FROM Members WHERE age > 30 LIMIT 30;)

Even translate the explanation to a different language on the fly:

In [9]:
locale = Symbol(answer_doc).translate('German')
display(locale)

<class 'botdyn.symbol.Symbol'>(value=Diese Abfrage wählt alle Daten aus der Mitgliedertabelle aus, in der das Alter größer als 30 ist.)

Fixing the query on the fly if something goes wrong or the user quickly wants to adapt a query:

In [21]:
sql.update(feedback="""Explanation: Never allow SELECT *, always use LIMIT to a max of x <= 50 entries, where x is the user specified limit.""");

In [22]:
res = sql('Select all users above the age of 30 from the Members table.')
display(res)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30 LIMIT 50)

In [23]:
sql.clear();

In [24]:
res = sql('Select all users above the age of 30 from the Members table.')
display(res)

<class 'examples.sql.SQL'>(value=SELECT * FROM Members WHERE age > 30)